In [1]:
import sqlite3
import pandas as pd

In [2]:


# Connect to the SQLite database
conn = sqlite3.connect("Auto_Reply_v3.db")

# Load Q&A pairs (assuming table name is 'All_Reviews')
query1 = 'SELECT "translation" AS Review, "Developer Reply Text" AS Reply FROM All_Reviews WHERE "Developer Reply Text" IS NOT NULL'
query2 = 'SELECT "translation" AS Review, "Reply" FROM "Auto Reply Hadith" WHERE "Reply" IS NOT NULL'
query3 = 'SELECT "translation" AS Review, "Reply" FROM Auto_Reply_Reviews WHERE "Reply" IS NOT NULL'
query4 = 'SELECT "translation" AS Review, "Reply" FROM Auto_Reply_Reviews_Al_Quran WHERE "Reply" IS NOT NULL'

df1 = pd.read_sql_query(query1, conn)
df2 = pd.read_sql_query(query2, conn)
df3 = pd.read_sql_query(query3, conn)
df4 = pd.read_sql_query(query4, conn)

df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Close connection
conn.close()

# Check data
df.head()

# craete a csv file 
# df.to_csv("Auto_Reply.csv", index=False)

,Review,Reply
0,The recent edition does explain the source of ...,BarakAllahu feekum for your kind words and sup...
1,Alhamdulillah very good app but it would have ...,Assalamu 'alaykum dear user. We would love to ...
2,"Alhamdulillah, this is a amazing app.",BarakAllahu feekum for your kind words and sup...
3,Very good app. But the word-by-word translatio...,"Assalamu 'alaykum, exciting news! French Word-..."
4,The app is really great!,Assalamu 'alaykum dear user. We would love to ...


Embedding Reviews

In [3]:
from sentence_transformers import SentenceTransformer

/home/tahlilmahfuz/Thesis_Implementations/RAG_Implementation/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = SentenceTransformer("all-MiniLM-L6-v2")

Create Faiss Vector Store

In [5]:
import faiss

In [6]:
def create_faiss_index(df):
    """Creates FAISS index for fast similarity search."""
    embeddings = model.encode(df["Review"].tolist(), convert_to_numpy=True)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index, embeddings



In [7]:
index, embeddings = create_faiss_index(df)

Retrieve the best match

In [8]:
def retrieve_reply(query, index, df, top_k=1):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)
    return df.iloc[indices[0][0]]['Reply']

In [21]:
query = "Best app of all time. Thank you so much. Keep doing it."
response = retrieve_reply(query, index, df)
print("Auto Reply:", response)

Auto Reply: BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and share with others in sha Allah!


In [20]:
query = "Worst experience of all time"
response = retrieve_reply(query, index, df)
print("Auto Reply:", response)

Auto Reply: BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and share with others in sha Allah!
